# Analyzing replicability of connectivity-based multivariate BWAS on the Human Connectome Project dataset

## Imports

In [80]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.model_selection import KFold, train_test_split, cross_val_predict, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from joblib import Parallel, delayed
from mlxtend.evaluate import permutation_test
sns.set(rc={"figure.figsize":(4, 2)})
sns.set_style("whitegrid")

## Load HCP data

We load functional network matrices (netmats) from the HCP1200-release, as published on connectomeDB: https://db.humanconnectome.org/
Due to licensoing issues, data is not supplied with the repository, but can be downloaded from the ConnectomeDB.
See [hcp_data/readme.md](hcp_data/readme.md) for more details.

In [81]:
# HCP data can be obtained from the connectomeDB
# data is not part of this repository
subjectIDs = pd.read_csv('hcp_data/subjectIDs.txt', header=None)

netmats_pearson = pd.read_csv('hcp_data/netmats1_correlationZ.txt',
                             sep=' ',
                             header=None)
netmats_pearson['ID'] = subjectIDs[0]
netmats_pearson.set_index('ID', drop=True, inplace=True)


netmats_parcor = pd.read_csv('hcp_data/netmats2_partial-correlation.txt',
                             sep=' ',
                             header=None)
netmats_parcor['ID'] = subjectIDs[0]
netmats_parcor.set_index('ID', drop=True, inplace=True)

behavior = pd.read_csv('hcp_data/hcp1200_behavioral_data.csv')
behavior = behavior.set_index('Subject', drop=True)

# convert age to numeric
age = []
for s in behavior['Age']:
    if s == '36+':
        age.append(36)
    else:
        split = s.split(sep='-')
        age.append(np.mean((float(split[0]), float(split[1]))))

behavior['age'] = age
behavior

,Release,Acquisition,Gender,Age,3T_Full_MR_Compl,T1_Count,T2_Count,3T_RS-fMRI_Count,3T_RS-fMRI_PctCompl,3T_Full_Task_fMRI,...,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final,age
Subject,,,,,,,,,,,,,,,,,,,,,
100004,S900,Q06,M,22-25,False,0,0,0,0.0,False,...,101.12,86.45,2.0,45.9,107.17,105.31,1.80,0.0,1.80,23.5
100206,S900,Q11,M,26-30,True,1,1,4,100.0,True,...,108.79,97.19,1.0,49.7,72.63,72.03,1.84,0.0,1.84,28.0
100307,Q1,Q01,F,26-30,True,1,1,4,100.0,True,...,101.12,86.45,0.0,38.6,71.69,71.76,1.76,0.0,1.76,28.0
100408,Q3,Q03,M,31-35,True,1,1,4,100.0,True,...,108.79,98.04,2.0,52.6,114.01,113.59,1.76,2.0,1.68,33.0
100610,S900,Q08,M,26-30,True,2,1,4,100.0,True,...,122.25,110.45,0.0,38.6,84.84,85.31,1.92,1.0,1.88,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992774,Q2,Q02,M,31-35,True,2,2,4,100.0,True,...,122.25,111.41,4.0,50.1,107.17,103.55,1.76,0.0,1.76,33.0
993675,S900,Q09,F,26-30,True,2,2,4,100.0,True,...,122.25,110.45,0.0,38.6,84.07,84.25,1.80,1.0,1.76,28.0
994273,S500,Q06,M,26-30,True,1,1,4,100.0,True,...,122.25,111.41,7.0,63.8,110.65,109.73,1.80,1.0,1.76,28.0


# Function to prepare target variable


In [98]:
def create_data(target='CogTotalComp_AgeAdj', feature_data=netmats_parcor):
    # it's a good practice to use pandas for merging, messing up subject order can be painful
    features = feature_data.columns
    df = behavior
    df = df.merge(feature_data, left_index=True, right_index=True, how='left')
    if target:
        df = df.dropna(subset = [target] + features.values.tolist())
        y = df[target].values
    else:
        df = df.dropna(subset = features.values.tolist())
        rng = np.random.default_rng(42)
        y = rng.normal(0,1, len(df))
    X = df[features].values
    return X, y

# Function implementing a single bootstrap iteration

We define a workhorse function which:
- randomly samples the discovery and the replication datasets,
- creates cross-validated estimates of predictive performance within the discovery sample
- finalizes the model by fitting it to the whole discovery sample (overfits the discovery but not the replication sample)
- use it to predict the replication sample

In [83]:
def bootstrap_workhorse(X, y, sample_size, model, random_state):

    #create discovery and replication samples by random sampling from the whole dataset (without replacement)
    X_discovery, X_replication, y_discovery, y_replication = train_test_split(X, y, train_size=sample_size, test_size=sample_size, shuffle=True, random_state=random_state)

    # obtain cross-validated predictions in the discovery sample
    predicted_discovery_cv = cross_val_predict(estimator=model, X=X_discovery, y=y_discovery, cv=10, n_jobs=1)
    # correlation between the cross-validated predictions and observations in the discovery sample
    # this is the correct, unbiased estimate!
    r_disc_cv = np.corrcoef(predicted_discovery_cv, y_discovery)[0, 1]
    # finalize model by training it on the full discovery sample (without cross-validation)
    final_model = model.fit(X=X_discovery, y=y_discovery)
    # obtain predictions with the final model on the discovery sample, note that this model actually overfits this sample.
    # we do this only to demonstrate biased estimates
    predicted_discovery_overfit = final_model.predict(X=X_discovery)
    # here we obtain the biased effect size (r) estimates for demonstrational purposes
    r_disc_overfit = np.corrcoef(predicted_discovery_overfit, y_discovery)[0, 1]

    # We use the final model to predict the replication sample
    # This is correct (no overfitting here), the final model did not see this data during training
    predicted_replication = final_model.predict(X=X_replication)
    # we obtain the out-of-sample prediction performance estimates
    r_rep = np.corrcoef(predicted_replication, y_replication)[0, 1]

    # below we calculate permutation-based p-values for all three effect size estimates (in-sample unbiased, in-sample biased, out-of-sample)
    # (one sided tests, testing for positive correlation)
    p_disc_cv = permutation_test(predicted_discovery_cv, y_discovery, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)
    p_disc_overfit = permutation_test(predicted_discovery_overfit, y_discovery, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)
    p_rep = permutation_test(predicted_replication, y_replication, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)
    # return results
    return r_disc_cv, r_disc_overfit, r_rep, p_disc_cv, p_disc_overfit, p_rep

All set, now we start the analysis.

# Replicability with sample sizes n=50, 100, 200, 300 and max
Here we train a few different models on 100 bootstrap samples.

We aggregate the results of our workhorse function in `n_bootstrap`=100 bootstrap cases (run in parallel).

The whole process is repeated for all sample sizes, fetaure_sets and target variables.

## Here we test age and 5 cognitive variables, including 'cognitive ability' (the main target variable in the target paper)
- age: age group of the participants
- CogTotalComp_AgeAdj: total cognitive ability
- PMAT24_A_CR, : Fluid Intelligence (Penn Progressive Matrices)
- CardSort_AgeAdj: Executive Function/Cognitive Flexibility (Dimensional Change Card Sort)
- Flanker_AgeAdj: Executive Function/Inhibition (Flanker Task)
- PicSeq_AgeAdj: Episodic Memory (Picture Sequence Memory)

# Reproducing the PCA+SVR-based model from the target paper
### Like in the target paper:
- Both PCA and SVR are done inside the cross-validation
- PCA reatains the firts k principal components that together explain 50% of the variance
- scikit-learn makes sure that PCA is only fit for the training samples
- both for the test sets (in the cross-validation) and the replication sample PCA is not re-fit, bt features are simply transformed with the already fit PCA

In [87]:
%%time

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'PCA_SVR': Pipeline([('pca', PCA(n_components=0.5)),
                         ('svr', SVR())])

}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set])

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })
                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print(tmp_data_frame.r_discovery_cv.mean(), tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res_PCA_SVR.csv')

df

*****************************************************************
netmats_parcor PCA_SVR age 50
0.01897346554269831 0.18901378266057708
Replicability at alpha = 0.05 : 57.14285714285714 %
Replicability at alpha = 0.01 : 14.285714285714285 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR age 100
0.18664851524294415 0.27347377929445293
Replicability at alpha = 0.05 : 89.55223880597015 %
Replicability at alpha = 0.01 : 40.298507462686565 %
Replicability at alpha = 0.005 : 31.343283582089555 %
Replicability at alpha = 0.001 : 11.940298507462686 %
*****************************************************************
netmats_parcor PCA_SVR age 200
0.3121257047522771 0.36175079133098437
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 98.9795918367347 %
Replicability at alpha = 0.005 : 95.91836734693877 %
Replicability at alpha = 0.001 : 80.61224489795919 

<timed exec>:60: RuntimeWarning: invalid value encountered in long_scalars


-0.039553381304751625 0.21262859874092135
Replicability at alpha = 0.05 : 90.9090909090909 %
Replicability at alpha = 0.01 : 45.45454545454545 %
Replicability at alpha = 0.005 : 18.181818181818183 %
Replicability at alpha = 0.001 : 9.090909090909092 %
*****************************************************************
netmats_parcor PCA_SVR CogTotalComp_AgeAdj 200
0.14966202410710186 0.2713589974517626
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 61.29032258064516 %
Replicability at alpha = 0.005 : 53.2258064516129 %
Replicability at alpha = 0.001 : 27.419354838709676 %
*****************************************************************
netmats_parcor PCA_SVR CogTotalComp_AgeAdj 300
0.23474749556135074 0.3101363215395622
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 94.73684210526315 %
Replicability at alpha = 0.005 : 94.73684210526315 %
Replicability at alpha = 0.001 : 76.84210526315789 %
************************************************

,connectivity,model,target,n,r_discovery_cv,r_discovery_overfit,r_replication,p_discovery_cv,p_discovery_overfit,p_replication
0,netmats_parcor,PCA_SVR,age,50,-0.353027,0.774355,0.204078,0.997003,0.000999,0.091908
1,netmats_parcor,PCA_SVR,age,50,0.081894,0.913355,0.360970,0.275724,0.000999,0.001998
2,netmats_parcor,PCA_SVR,age,50,-0.258248,0.841646,0.265158,0.962038,0.000999,0.025974
3,netmats_parcor,PCA_SVR,age,50,0.050106,0.899457,-0.206159,0.374625,0.000999,0.923077
4,netmats_parcor,PCA_SVR,age,50,0.130115,0.892774,0.282873,0.196803,0.000999,0.026973
...,...,...,...,...,...,...,...,...,...,...
5995,netmats_pearson,PCA_SVR,PicSeq_AgeAdj,501,0.125791,0.360809,0.062856,0.004995,0.000999,0.085914
5996,netmats_pearson,PCA_SVR,PicSeq_AgeAdj,501,0.060623,0.352693,0.142834,0.085914,0.000999,0.000999
5997,netmats_pearson,PCA_SVR,PicSeq_AgeAdj,501,0.084630,0.373386,0.163908,0.028971,0.000999,0.000999
5998,netmats_pearson,PCA_SVR,PicSeq_AgeAdj,501,0.071084,0.376163,0.109268,0.065934,0.000999,0.012987


# Now we fit a simple Ridge regression
(no feature selection, no hyperparameter optimization)
This can be expected to perform better on low samples than SVR.

In [84]:
%%time

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'ridge': Ridge()
}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set])

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })
                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print(tmp_data_frame.r_discovery_cv.mean(), tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res_ridge.csv')

df


*****************************************************************
netmats_parcor ridge age 50
0.17791423402156398 0.2609198136325508
Replicability at alpha = 0.05 : 58.536585365853654 %
Replicability at alpha = 0.01 : 14.634146341463413 %
Replicability at alpha = 0.005 : 12.195121951219512 %
Replicability at alpha = 0.001 : 7.317073170731707 %
*****************************************************************
netmats_parcor ridge age 100
0.3076875117666388 0.34287580012326385
Replicability at alpha = 0.05 : 97.75280898876404 %
Replicability at alpha = 0.01 : 71.91011235955057 %
Replicability at alpha = 0.005 : 58.42696629213483 %
Replicability at alpha = 0.001 : 38.20224719101123 %
*****************************************************************
netmats_parcor ridge age 200
0.3896290546757868 0.4213171713707975
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 100.0 %
Replicability at alpha = 0.005 : 99.0 %
Replicability at alpha = 0.001 : 97.0 %
*****************

,connectivity,model,target,n,r_discovery_cv,r_discovery_overfit,r_replication,p_discovery_cv,p_discovery_overfit,p_replication
0,netmats_parcor,ridge,age,50,-0.048518,1.0,0.404661,0.629371,0.000999,0.002997
1,netmats_parcor,ridge,age,50,0.111210,1.0,0.281041,0.210789,0.000999,0.026973
2,netmats_parcor,ridge,age,50,0.103342,1.0,0.107181,0.227772,0.000999,0.216783
3,netmats_parcor,ridge,age,50,0.255112,1.0,-0.203981,0.047952,0.000999,0.909091
4,netmats_parcor,ridge,age,50,0.425614,1.0,0.385665,0.001998,0.000999,0.004995
...,...,...,...,...,...,...,...,...,...,...
5995,netmats_pearson,ridge,PicSeq_AgeAdj,501,0.209622,1.0,0.217841,0.000999,0.000999,0.000999
5996,netmats_pearson,ridge,PicSeq_AgeAdj,501,0.237770,1.0,0.201980,0.000999,0.000999,0.000999
5997,netmats_pearson,ridge,PicSeq_AgeAdj,501,0.268639,1.0,0.203847,0.000999,0.000999,0.000999
5998,netmats_pearson,ridge,PicSeq_AgeAdj,501,0.173644,1.0,0.195714,0.001998,0.000999,0.000999


# Null scenario with random target
To evaluate false positives with biased estimates

In [99]:
%%time

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor
}

models = {
    'PCA_SVR': Pipeline([('pca', PCA(n_components=0.5)),
                         ('svr', SVR())])

}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in [None]:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set]) # gives a random y when target is None

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })
                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print(tmp_data_frame.r_discovery_cv.mean(), tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res_null_PCA_SVR.csv')

df

*****************************************************************
netmats_parcor PCA_SVR None 50
-0.10752243410278593 0.015330494496431592
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR None 100
-0.05119164595574469 0.018628014156943713
Replicability at alpha = 0.05 : 14.285714285714285 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR None 200
-0.026918006714795394 -0.001351453184769438
Replicability at alpha = 0.05 : 0.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor PCA_SVR None 300
-0.0

,connectivity,model,target,n,r_discovery_cv,r_discovery_overfit,r_replication,p_discovery_cv,p_discovery_overfit,p_replication
0,netmats_parcor,PCA_SVR,None,50,0.019118,0.893737,0.062928,0.445554,0.000999,0.330669
1,netmats_parcor,PCA_SVR,None,50,-0.319547,0.897756,0.134277,0.995005,0.000999,0.185814
2,netmats_parcor,PCA_SVR,None,50,-0.120632,0.871983,0.052140,0.816184,0.000999,0.344655
3,netmats_parcor,PCA_SVR,None,50,-0.224169,0.814265,-0.009114,0.938062,0.000999,0.536464
4,netmats_parcor,PCA_SVR,None,50,-0.175718,0.889370,0.083803,0.876124,0.000999,0.279720
...,...,...,...,...,...,...,...,...,...,...
495,netmats_parcor,PCA_SVR,None,501,-0.071740,0.883689,0.026235,0.950050,0.000999,0.282717
496,netmats_parcor,PCA_SVR,None,501,0.009939,0.877526,0.019956,0.414585,0.000999,0.338661
497,netmats_parcor,PCA_SVR,None,501,-0.009945,0.875216,0.015069,0.608392,0.000999,0.349650
498,netmats_parcor,PCA_SVR,None,501,-0.020732,0.876886,-0.082211,0.686314,0.000999,0.969031


-----------------------------------------------------------------------
# The main code ends here.
Below we perform some exploratory analysis with other models, for the sake of curiosity.
This doesn't need to be revied.

# SVR, without PCA

In [86]:
%%time

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'SVR': SVR()
}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set])

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })
                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print(tmp_data_frame.r_discovery_cv.mean(), tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res_SVR.csv')

df

*****************************************************************
netmats_parcor SVR age 50
-0.01643376964637595 0.23423014405557027
Replicability at alpha = 0.05 : 45.45454545454545 %
Replicability at alpha = 0.01 : 9.090909090909092 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor SVR age 100
0.2004971727400091 0.31977467957054306
Replicability at alpha = 0.05 : 95.52238805970148 %
Replicability at alpha = 0.01 : 49.25373134328358 %
Replicability at alpha = 0.005 : 43.28358208955223 %
Replicability at alpha = 0.001 : 13.432835820895523 %
*****************************************************************
netmats_parcor SVR age 200
0.34526027745129295 0.40305868339855644
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 98.98989898989899 %
Replicability at alpha = 0.005 : 94.94949494949495 %
Replicability at alpha = 0.001 : 88.88888888888889 %
***********

<timed exec>:58: RuntimeWarning: invalid value encountered in long_scalars


-0.14562995074992818 0.24088134739904876
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor SVR CogTotalComp_AgeAdj 200
0.03603161832397271 0.2851671994147706
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 43.47826086956522 %
Replicability at alpha = 0.005 : 34.78260869565217 %
Replicability at alpha = 0.001 : 8.695652173913043 %
*****************************************************************
netmats_parcor SVR CogTotalComp_AgeAdj 300
0.1468259280416723 0.309336370533758
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 71.23287671232876 %
Replicability at alpha = 0.005 : 67.12328767123287 %
Replicability at alpha = 0.001 : 49.31506849315068 %
*****************************************************************
netmats_parcor SVR CogTotalComp_AgeAdj max
0.25

,connectivity,model,target,n,r_discovery_cv,r_discovery_overfit,r_replication,p_discovery_cv,p_discovery_overfit,p_replication
0,netmats_parcor,SVR,age,50,-0.330564,0.899135,0.288430,0.994006,0.000999,0.029970
1,netmats_parcor,SVR,age,50,-0.023555,0.963181,0.291646,0.591409,0.000999,0.019980
2,netmats_parcor,SVR,age,50,-0.176967,0.968481,0.092822,0.884116,0.000999,0.244755
3,netmats_parcor,SVR,age,50,0.093535,0.960486,-0.161212,0.299700,0.000999,0.854146
4,netmats_parcor,SVR,age,50,0.220397,0.958377,0.310812,0.072927,0.000999,0.016983
...,...,...,...,...,...,...,...,...,...,...
5995,netmats_pearson,SVR,PicSeq_AgeAdj,501,0.167868,0.469294,0.119368,0.000999,0.000999,0.004995
5996,netmats_pearson,SVR,PicSeq_AgeAdj,501,0.134985,0.493440,0.222072,0.001998,0.000999,0.000999
5997,netmats_pearson,SVR,PicSeq_AgeAdj,501,0.155819,0.522091,0.223508,0.000999,0.000999,0.000999
5998,netmats_pearson,SVR,PicSeq_AgeAdj,501,0.065164,0.484062,0.215828,0.081918,0.000999,0.000999


# Ridge, but with PCA

In [88]:
%%time

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'PCA_Ridge': Pipeline([('pca', PCA(n_components=0.5)),
                         ('ridge', Ridge())])

}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set])

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })
                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print(tmp_data_frame.r_discovery_cv.mean(), tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res_PCA_Ridge.csv')

df

*****************************************************************
netmats_parcor PCA_Ridge age 50
0.12862642959256065 0.2154842766278765
Replicability at alpha = 0.05 : 42.42424242424242 %
Replicability at alpha = 0.01 : 9.090909090909092 %
Replicability at alpha = 0.005 : 9.090909090909092 %
Replicability at alpha = 0.001 : 3.0303030303030303 %
*****************************************************************
netmats_parcor PCA_Ridge age 100
0.25477086132155136 0.31779871366485796
Replicability at alpha = 0.05 : 92.40506329113924 %
Replicability at alpha = 0.01 : 55.69620253164557 %
Replicability at alpha = 0.005 : 49.36708860759494 %
Replicability at alpha = 0.001 : 20.253164556962027 %
*****************************************************************
netmats_parcor PCA_Ridge age 200
0.35627669437007353 0.39305369552769015
Replicability at alpha = 0.05 : 100.0 %
Replicability at alpha = 0.01 : 97.9381443298969 %
Replicability at alpha = 0.005 : 96.90721649484536 %
Replicability at al

,connectivity,model,target,n,r_discovery_cv,r_discovery_overfit,r_replication,p_discovery_cv,p_discovery_overfit,p_replication
0,netmats_parcor,PCA_Ridge,age,50,-0.260636,0.457983,0.225977,0.969031,0.001998,0.062937
1,netmats_parcor,PCA_Ridge,age,50,0.189752,0.687611,0.246691,0.098901,0.000999,0.049950
2,netmats_parcor,PCA_Ridge,age,50,-0.014329,0.730443,-0.045666,0.538462,0.000999,0.601399
3,netmats_parcor,PCA_Ridge,age,50,0.194567,0.710963,-0.162378,0.101898,0.000999,0.861139
4,netmats_parcor,PCA_Ridge,age,50,0.223428,0.717670,0.304525,0.075924,0.000999,0.015984
...,...,...,...,...,...,...,...,...,...,...
5995,netmats_pearson,PCA_Ridge,PicSeq_AgeAdj,501,0.174858,0.286206,0.086955,0.000999,0.000999,0.029970
5996,netmats_pearson,PCA_Ridge,PicSeq_AgeAdj,501,0.114124,0.261799,0.146330,0.006993,0.000999,0.000999
5997,netmats_pearson,PCA_Ridge,PicSeq_AgeAdj,501,0.193151,0.310800,0.164787,0.000999,0.000999,0.000999
5998,netmats_pearson,PCA_Ridge,PicSeq_AgeAdj,501,0.144112,0.285125,0.095999,0.002997,0.000999,0.018981


# KernelRidge, just for fun
(I guess for non-linear kernel ridge we really need thousands of participants)

In [89]:
%%time

from sklearn.kernel_ridge import KernelRidge

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'KernelRidge': KernelRidge()

}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set])

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })
                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print(tmp_data_frame.r_discovery_cv.mean(), tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res_KernelRidge.csv')

df

*****************************************************************
netmats_parcor KernelRidge age 50
-0.057193958649027946 -0.00987408448972495
Replicability at alpha = 0.05 : 50.0 %
Replicability at alpha = 0.01 : 0.0 %
Replicability at alpha = 0.005 : 0.0 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor KernelRidge age 100
0.09658448630512596 0.13168896630686272
Replicability at alpha = 0.05 : 42.857142857142854 %
Replicability at alpha = 0.01 : 14.285714285714285 %
Replicability at alpha = 0.005 : 7.142857142857142 %
Replicability at alpha = 0.001 : 0.0 %
*****************************************************************
netmats_parcor KernelRidge age 200
0.2614177407654067 0.28952399198813356
Replicability at alpha = 0.05 : 98.93617021276596 %
Replicability at alpha = 0.01 : 91.48936170212765 %
Replicability at alpha = 0.005 : 88.29787234042553 %
Replicability at alpha = 0.001 : 61.702127659574465 %
************

,connectivity,model,target,n,r_discovery_cv,r_discovery_overfit,r_replication,p_discovery_cv,p_discovery_overfit,p_replication
0,netmats_parcor,KernelRidge,age,50,-0.289530,1.0,0.097963,0.981019,0.000999,0.253746
1,netmats_parcor,KernelRidge,age,50,-0.418438,1.0,0.020128,0.999001,0.000999,0.449550
2,netmats_parcor,KernelRidge,age,50,-0.141016,1.0,-0.140765,0.822178,0.000999,0.838162
3,netmats_parcor,KernelRidge,age,50,0.131642,1.0,-0.102626,0.183816,0.000999,0.760240
4,netmats_parcor,KernelRidge,age,50,0.024683,1.0,0.203918,0.426573,0.000999,0.079920
...,...,...,...,...,...,...,...,...,...,...
5995,netmats_pearson,KernelRidge,PicSeq_AgeAdj,501,0.202720,1.0,0.225979,0.000999,0.000999,0.000999
5996,netmats_pearson,KernelRidge,PicSeq_AgeAdj,501,0.223386,1.0,0.217658,0.000999,0.000999,0.000999
5997,netmats_pearson,KernelRidge,PicSeq_AgeAdj,501,0.274774,1.0,0.206499,0.000999,0.000999,0.000999
5998,netmats_pearson,KernelRidge,PicSeq_AgeAdj,501,0.166591,1.0,0.203236,0.000999,0.000999,0.000999


More complex models are not reviewd here, as KernelRidge doesn't seem to add a lot.

*See the notebook called 'plot_results.ipynb' for more detail.*